# Tarea 1

In [7]:
# Instalación de librerías necesarias
!pip install rdflib pandas

import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS

import io
from google.colab import drive
drive.mount('/content/drive')



MessageError: Error: credential propagation was unsuccessful

In [3]:
# Ruta al archivo en Google Drive
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Datos abiertos/empleos_turisticos_region_de_valparaiso.csv'

In [4]:
# Cargar el archivo CSV
df = pd.read_csv(csv_file_path, encoding='latin1', delimiter=';')

In [5]:
# Mostrar las primeras filas para verificar el contenido
print("Primeras filas del dataset:")
print(df.head())

# Generar un resumen estadístico del dataset
print("\nResumen del dataset:")
df_summary = df.describe(include='all')  # Incluye estadísticas para todas las columnas
print(df_summary)

Primeras filas del dataset:
   Id_Encuesta  Actividad  Mz  Act_Cod  Reg            A1  \
0          169          1   2       14    5  ALIMENTACION   
1          170          1   2       14    5  ALIMENTACION   
2          171          1   2       14    5  ALIMENTACION   
3          172          1   2       14    5  ALIMENTACION   
4          173          1   2       14    5  ALIMENTACION   

                         A2_1  A3_1              A2_2  A3_2  ... H5_10  \
0                     COMIDAS    70       ALOJAMIENTO  30.0  ...   0.0   
1           COMIDA (PESCADOS)    60         CAFETERIA  40.0  ...   0.0   
2  PESCADOR MARISCOS Y CARNES    70  EMPANADAS VARIAS  30.0  ...   0.0   
3                  RESTAURANT    85         CAFETERIA  15.0  ...   0.0   
4  COMIDA PESCADOS Y MARISCOS   100               NaN   NaN  ...   0.0   

  H5_10_Imp  H5_11 H5_11_Imp        H5_12  H5_12_Imp         H5_9esp  \
0         0    0.0         0    5192000.0    5192000  CONTRIBUCIONES   
1         0    0

In [6]:
# Selección de columnas clave
columns_to_use = ['Actividad', 'A1', 'A2_1']  # Asegúrate de que estas columnas existan en el archivo
df_selected = df[columns_to_use].dropna()

In [ ]:
# Validar si las columnas existen en el dataset
for col in columns_to_use:
    if col not in df.columns:
        print(f"La columna '{col}' NO existe en el dataset.")
    else:
        print(f"La columna '{col}' existe en el dataset.")


La columna 'Actividad' existe en el dataset.
La columna 'A1' existe en el dataset.
La columna 'A2_1' existe en el dataset.


In [7]:
# Crear un gráfico RDF
g = Graph()
print(g)

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


### ¿Por qué usamos namespaces en RDF?
Los namespaces en RDF sirven para identificar de manera única recursos y propiedades dentro de un modelo semántico. Nos permiten reutilizar vocabularios estándar y definir nuestros propios términos para evitar conflictos de nombres.

#### Namespaces usados:
1. **EX (http://example.org/empleos/):**  
   - Define recursos específicos del dominio, como actividades turísticas.
   - Ejemplo: `http://example.org/empleos/actividad/1`.

2. **SCHEMA (http://schema.org/):**  
   - Un vocabulario estándar ampliamente utilizado para describir entidades como nombres, categorías, y ubicaciones.
   - Propiedades usadas:
     - `schema:name`: Nombre de la actividad.
     - `schema:category`: Categoría o tipo de actividad.
     - `schema:location`: Ubicación o región.

#### Ejemplo en Turtle:
```turtle
ex:actividad/1 a schema:Thing ;
    schema:name "Restaurante" ;
    schema:category "Comida" ;
    schema:location "Valparaíso" .


In [4]:
# Crear un gráfico RDF en memoria
g = Graph()

# Cargar el archivo Turtle generado
g.parse(output_file, format="turtle")

# Ejemplo de consulta SPARQL: Obtener todas las actividades
query_1 = """
    SELECT ?actividad ?name ?category ?location
    WHERE {
        ?actividad a <http://schema.org/Thing> .
        ?actividad <http://schema.org/name> ?name .
        ?actividad <http://schema.org/category> ?category .
        ?actividad <http://schema.org/location> ?location .
    }
"""

# Ejecutar la consulta
results_1 = g.query(query_1)

# Mostrar los resultados
for row in results_1:
    print(f"Actividad: {row['actividad']}, Nombre: {row['name']}, Categoría: {row['category']}, Ubicación: {row['location']}")


NameError: name 'Graph' is not defined

In [9]:
# Transformar filas del dataset a triples RDF
for index, row in df_selected.iterrows():
    actividad_uri = URIRef(EX[f"actividad/{index}"])
    g.add((actividad_uri, RDF.type, SCHEMA.Thing))
    g.add((actividad_uri, SCHEMA.name, Literal(row['A2_1'])))
    g.add((actividad_uri, SCHEMA.category, Literal(row['Actividad'])))
    g.add((actividad_uri, SCHEMA.location, Literal(row['A1'])))

In [10]:
# Mostrar un ejemplo de los triples generados en formato Turtle
example_output = g.serialize(format="turtle") # Remove .decode("utf-8")
# Mostrar las primeras líneas
print("\n".join(example_output.splitlines()[:20]))

@prefix schema1: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/empleos/actividad/0> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "COMIDAS" .

<http://example.org/empleos/actividad/1> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "COMIDA (PESCADOS)" .

<http://example.org/empleos/actividad/10> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "PESCADOS, MARISCOS" .

<http://example.org/empleos/actividad/100> a schema1:Thing ;
    schema1:category 1 ;


In [11]:
# Guardar el gráfico RDF en un archivo Turtle en Google Drive
output_file = '/content/drive/My Drive/empleos_turisticos.ttl'  # Cambia esta ruta si prefieres otro lugar
g.serialize(destination=output_file, format="turtle")
print(f"Archivo RDF guardado en Google Drive como {output_file}.")

Archivo RDF guardado en Google Drive como /content/drive/My Drive/empleos_turisticos.ttl.


In [12]:
# Mostrar un ejemplo de los triples generados
print(g.serialize(format="turtle")) # Remove .decode("utf-8")

@prefix schema1: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/empleos/actividad/0> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "COMIDAS" .

<http://example.org/empleos/actividad/1> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "COMIDA (PESCADOS)" .

<http://example.org/empleos/actividad/10> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "PESCADOS, MARISCOS" .

<http://example.org/empleos/actividad/100> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "ALMUERZOS" .

<http://example.org/empleos/actividad/101> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "ALIMENTACION" ;
    schema1:name "COMPLETOS BEBIDAS" .

<http://example.org/empleos/actividad/102> a schema1:Thing ;
    schema1:category 1 ;
    schema1:location "AL

Servidor Flask corriendo en: http://localhost:5000/sparql
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

# Tarea 2


In [10]:
# Instalación de librerías necesarias
!pip install rdflib pandas

import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS

import io
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
# Instalación de librerías necesarias
!pip install rdflib pandas

import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS

import io
from google.colab import drive
drive.mount('/content/drive')

# Ruta al archivo en Google Drive
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/Datos abiertos/empleos_turisticos_region_de_valparaiso.csv'

# Cargar el archivo CSV
df = pd.read_csv(csv_file_path, encoding='latin1', delimiter=';') # Define df here by reading the CSV file

# Selección de columnas clave
columns_to_use = ['Actividad', 'A1', 'A2_1']  # Asegúrate de que estas columnas existan en el archivo
df_selected = df[columns_to_use].dropna()

# Validar si las columnas existen en el dataset
for col in columns_to_use:
    if col not in df.columns:
        print(f"La columna '{col}' NO existe en el dataset.")
    else:
        print(f"La columna '{col}' existe en el dataset.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
La columna 'Actividad' existe en el dataset.
La columna 'A1' existe en el dataset.
La columna 'A2_1' existe en el dataset.


In [13]:
# Crear un gráfico RDF
g = Graph()

# Definir namespaces
EX = Namespace("http://example.org/empleos/")
SCHEMA = Namespace("http://schema.org/")
g.bind("ex", EX)
g.bind("schema", SCHEMA)

# Transformar filas del dataset a triples RDF
for index, row in df_selected.iterrows():
    actividad_uri = URIRef(EX[f"actividad/{index}"])
    g.add((actividad_uri, RDF.type, SCHEMA.Thing))
    g.add((actividad_uri, SCHEMA.name, Literal(row['A2_1'])))
    g.add((actividad_uri, SCHEMA.category, Literal(row['Actividad'])))
    g.add((actividad_uri, SCHEMA.location, Literal(row['A1'])))

# Guardar el gráfico RDF en un archivo Turtle en Google Drive
output_file = '/content/drive/My Drive/empleos_turisticos.ttl'  # Cambia esta ruta si prefieres otro lugar
g.serialize(destination=output_file, format="turtle")
print(f"Archivo RDF guardado en Google Drive como {output_file}.")


Archivo RDF guardado en Google Drive como /content/drive/My Drive/empleos_turisticos.ttl.


In [14]:
# Crear un gráfico RDF
g = Graph()

# Definir namespaces
EX = Namespace("http://example.org/empleos/")
SCHEMA = Namespace("http://schema.org/")
g.bind("ex", EX)
g.bind("schema", SCHEMA)

# Transformar filas del dataset a triples RDF
for index, row in df_selected.iterrows():
    actividad_uri = URIRef(EX[f"actividad/{index}"])
    g.add((actividad_uri, RDF.type, SCHEMA.Thing))
    g.add((actividad_uri, SCHEMA.name, Literal(row['A2_1'])))
    g.add((actividad_uri, SCHEMA.category, Literal(row['Actividad'])))
    g.add((actividad_uri, SCHEMA.location, Literal(row['A1'])))

# Guardar el gráfico RDF en un archivo Turtle en Google Drive
output_file = '/content/drive/My Drive/empleos_turisticos.ttl'  # Cambia esta ruta si prefieres otro lugar
g.serialize(destination=output_file, format="turtle")
print(f"Archivo RDF guardado en Google Drive como {output_file}.")


Archivo RDF guardado en Google Drive como /content/drive/My Drive/empleos_turisticos.ttl.


In [15]:
# Ejemplo de consulta SPARQL: Obtener todas las actividades
query_1 = """
    SELECT ?actividad ?name ?category ?location
    WHERE {
        ?actividad a <http://schema.org/Thing> .
        ?actividad <http://schema.org/name> ?name .
        ?actividad <http://schema.org/category> ?category .
        ?actividad <http://schema.org/location> ?location .
    }
"""

# Ejecutar la consulta
results_1 = g.query(query_1)

# Mostrar los resultados
for row in results_1:
    print(f"Actividad: {row['actividad']}, Nombre: {row['name']}, Categoría: {row['category']}, Ubicación: {row['location']}")


Actividad: http://example.org/empleos/actividad/0, Nombre: COMIDAS, Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/1, Nombre: COMIDA (PESCADOS), Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/2, Nombre: PESCADOR MARISCOS Y CARNES, Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/3, Nombre: RESTAURANT, Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/4, Nombre: COMIDA PESCADOS Y MARISCOS, Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/5, Nombre: VENTA DE COMIDA, Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/6, Nombre: VERNTA DE BEBIDAS ALCOHOLICAS, Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/7, Nombre: VENTA DE COMIDA, Categoría: 1, Ubicación: ALIMENTACION
Actividad: http://example.org/empleos/actividad/8, Nombre: COMI

In [17]:
!pip install SPARQLWrapper  # Install the necessary library

In [18]:
# Instalación de librerías necesarias
!pip install SPARQLWrapper  # Install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON

# Endpoint de Wikidata
wikidata_endpoint = "https://query.wikidata.org/sparql"

# ... (rest of your code) ...

vamos a hacer una consulta SPARQL que obtenga las actividades y las relacione con entidades de Wikidata, obteniendo las etiquetas correspondientes de esas entidades.

```
## Esto tiene formato de código
```



In [27]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Endpoint de Wikidata
wikidata_endpoint = "https://query.wikidata.org/sparql"

# Consulta SPARQL federada combinando el dataset local con Wikidata
query_federada = """
    SELECT ?actividad ?name ?category ?location ?wikidata_label
    WHERE {
        ?actividad a <http://schema.org/Thing> .
        ?actividad <http://schema.org/name> ?name .
        ?actividad <http://schema.org/category> ?category .
        ?actividad <http://schema.org/location> ?location .

        # Consulta federada a Wikidata para obtener información adicional
        SERVICE <https://query.wikidata.org/sparql> {
            ?item rdfs:label ?wikidata_label .
            FILTER(LANG(?wikidata_label) = "en")
            ?item wdt:P31 wd:Q5 .  # Buscar solo entidades de tipo humano
        }
    }
"""




In [34]:

# Configuración del endpoint SPARQL
sparql = SPARQLWrapper(wikidata_endpoint)
sparql.setQuery(query_federada)
sparql.setReturnFormat(JSON)

# Ejecutar la consulta
results = sparql.query().convert()

# Verificar que hay resultados
if 'results' in results and 'bindings' in results['results']:
    # Iterar sobre los resultados
    for row in results["results"]["bindings"]:
        # Acceder a cada variable de la consulta como en el código anterior
        actividad = row['actividad']['value'] if 'actividad' in row else 'N/A'
        name = row['name']['value'] if 'name' in row else 'N/A'
        category = row['category']['value'] if 'category' in row else 'N/A'
        location = row['location']['value'] if 'location' in row else 'N/A'
        wikidata_label = row['wikidata_label']['value'] if 'wikidata_label' in row else 'N/A'

        # Imprimir los resultados como en el código anterior
        print(f"Actividad: {actividad}, Nombre: {name}, Categoría: {category}, "
              f"Ubicación: {location}, Wikidata Label: {wikidata_label}")
else:
    print("No se encontraron resultados en la consulta.")

In [38]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Endpoint de Wikidata
wikidata_endpoint = "https://query.wikidata.org/sparql"

# Consulta SPARQL federada combinando el dataset local con Wikidata
query_federada = """
    SELECT ?actividad ?name ?category ?location ?wikidata_label
    WHERE {
        ?actividad a <http://schema.org/Thing> .
        ?actividad <http://schema.org/name> ?name .
        ?actividad <http://schema.org/category> ?category .
        ?actividad <http://schema.org/location> ?location .

        # Consulta federada a Wikidata para obtener información adicional
        SERVICE <https://query.wikidata.org/sparql> {
            ?item rdfs:label ?wikidata_label .
            FILTER(LANG(?wikidata_label) = "en")
            ?item wdt:P31 wd:Q5 .  # Buscar solo entidades de tipo humano
        }
    }
"""

# Configuración del endpoint SPARQL
sparql = SPARQLWrapper(wikidata_endpoint)
sparql.setQuery(query_federada)
sparql.setReturnFormat(JSON)

# Ejecutar la consulta
results = sparql.query().convert()

# Imprimir la estructura completa de los resultados para diagnosticar
print("Estructura completa de los resultados:")
print(results)

# Verificar si los resultados tienen la clave 'results' y 'bindings'
if 'results' in results and 'bindings' in results['results']:
    # Iterar sobre los resultados
    for row in results["results"]["bindings"]:
        # Imprimir cada fila de resultados para ver cómo están estructurados
        print(row)  # Imprime cada fila de resultados para su inspección
else:
    print("No se encontraron resultados en la consulta.")


Estructura completa de los resultados:
{'head': {'vars': ['actividad', 'name', 'category', 'location', 'wikidata_label']}, 'results': {'bindings': []}}
